FOURTH AND FINAL STEP

    we load the data we have from Posters.npy and Genre.npy and train the CNN model

In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dropout
from keras.models import Sequential
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [2]:
loaded_Posters = np.load('Posters.npy')
loaded_Genres = np.load('Genre.npy')

In [3]:
loaded_Posters

array([[[[ 21,  20,  38],
         [ 23,  22,  40],
         [ 26,  25,  43],
         ...,
         [ 30,  28,  52],
         [ 34,  32,  56],
         [ 37,  35,  59]],

        [[ 15,  14,  32],
         [ 15,  14,  32],
         [ 16,  15,  33],
         ...,
         [ 38,  36,  60],
         [ 33,  31,  55],
         [ 28,  26,  50]],

        [[ 11,   9,  30],
         [ 10,   8,  29],
         [  8,   6,  27],
         ...,
         [ 39,  37,  59],
         [ 42,  40,  62],
         [ 45,  43,  65]],

        ...,

        [[ 10,   9,  23],
         [ 10,   9,  23],
         [  9,   8,  22],
         ...,
         [  2,   3,   7],
         [  3,   4,   8],
         [  3,   4,   8]],

        [[ 14,  13,  27],
         [ 14,  13,  27],
         [ 14,  13,  27],
         ...,
         [  1,   2,   6],
         [  2,   3,   7],
         [  2,   3,   7]],

        [[ 14,  13,  27],
         [ 15,  14,  28],
         [ 15,  14,  28],
         ...,
         [  3,   4,   8],
        

In [4]:
loaded_Genres

array(['Animation', 'Comedy', 'Drama', ..., 'Thriller', 'Drama', 'Horror'],
      dtype='<U32')